In [1]:
import logging

import probtorch
import torch

import combinators
import importance
import mcmc
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = combinators.ParticleTrace(1)

In [4]:
mu = torch.zeros(1)
sigma = torch.ones(1) / 2
delta = torch.ones(1)
zs = torch.ones(50+1) * -1
zs[0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)

In [6]:
for t in range(zs.shape[0] - 1):
    zs[t+1], mu, sigma, delta = ssm_step((zs[t], mu, sigma, delta), t, trace=generative)

In [7]:
data = generative.unwrap(lambda k, rv: 'X_' in k)

In [8]:
num_particles = 100

In [9]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(1.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [10]:
init_ssm_generative = combinators.Model(ssm.init_ssm, hyper=ssm_params)
smc_ssm = smc.SequentialMonteCarlo(combinators.Model(ssm.ssm_step), 50, initializer=init_ssm_generative)

In [11]:
init_ssm_proposal = combinators.Model(ssm.init_ssm, trainable=ssm_params)
ssm_step_proposal = combinators.Model(ssm.ssm_step)
ssm_proposal = combinators.Model.compose(combinators.Model.sequence(ssm_step_proposal, 50),
                                         init_ssm_proposal, intermediate_name='initializer')

In [12]:
ssm_importance = importance.ImportanceSampler(model=smc_ssm, proposal=ssm_proposal)

In [13]:
inference, init_ssm_params = smc.variational_smc(num_particles, ssm_importance, 1000, data,
                                                 use_cuda=True, lr=1e-1)

09/18/2018 12:06:33 Variational SMC ELBO=-2.75413666e+02 at epoch 1
09/18/2018 12:06:33 Variational SMC ELBO=-2.60561310e+02 at epoch 2
09/18/2018 12:06:34 Variational SMC ELBO=-2.14799164e+02 at epoch 3
09/18/2018 12:06:35 Variational SMC ELBO=-2.06225357e+02 at epoch 4
09/18/2018 12:06:35 Variational SMC ELBO=-2.32438507e+02 at epoch 5
09/18/2018 12:06:36 Variational SMC ELBO=-2.57670197e+02 at epoch 6
09/18/2018 12:06:36 Variational SMC ELBO=-2.13331909e+02 at epoch 7
09/18/2018 12:06:37 Variational SMC ELBO=-1.98092896e+02 at epoch 8
09/18/2018 12:06:38 Variational SMC ELBO=-2.28686752e+02 at epoch 9
09/18/2018 12:06:38 Variational SMC ELBO=-2.35545380e+02 at epoch 10
09/18/2018 12:06:39 Variational SMC ELBO=-2.44150009e+02 at epoch 11
09/18/2018 12:06:39 Variational SMC ELBO=-2.03846176e+02 at epoch 12
09/18/2018 12:06:40 Variational SMC ELBO=-2.39560059e+02 at epoch 13
09/18/2018 12:06:40 Variational SMC ELBO=-1.98779770e+02 at epoch 14
09/18/2018 12:06:41 Variational SMC ELBO=-1

In [14]:
for t in range(1, zs.shape[0]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 4.300280
SMC MSE at time 2: 0.042432
SMC MSE at time 3: 0.085995
SMC MSE at time 4: 0.775874
SMC MSE at time 5: 3.133495
SMC MSE at time 6: 0.078289
SMC MSE at time 7: 0.631686
SMC MSE at time 8: 0.334342
SMC MSE at time 9: 0.265827
SMC MSE at time 10: 0.125702
SMC MSE at time 11: 0.002956
SMC MSE at time 12: 0.745666
SMC MSE at time 13: 0.814802
SMC MSE at time 14: 1.007472
SMC MSE at time 15: 0.065814
SMC MSE at time 16: 0.852415
SMC MSE at time 17: 1.343045
SMC MSE at time 18: 1.175189
SMC MSE at time 19: 0.684706
SMC MSE at time 20: 4.310127
SMC MSE at time 21: 0.067410
SMC MSE at time 22: 0.271507
SMC MSE at time 23: 0.393144
SMC MSE at time 24: 2.094539
SMC MSE at time 25: 0.193701
SMC MSE at time 26: 0.000031
SMC MSE at time 27: 1.757690
SMC MSE at time 28: 0.037922
SMC MSE at time 29: 0.889509
SMC MSE at time 30: 2.304016
SMC MSE at time 31: 0.620106
SMC MSE at time 32: 0.090247
SMC MSE at time 33: 4.080547
SMC MSE at time 34: 1.894677
SMC MSE at time 35: 1.8

In [15]:
particle_mh = mcmc.IndependentMH(smc_ssm, ssm_proposal, 1000)

In [16]:
samples, elbos = particle_mh(trace=importance.ResamplerTrace(num_particles, data=data))
inference = particle_mh.trace

In [17]:
inference

Trace{'mu': Normal([torch.FloatTensor of size 100]), 'sigma': Normal([torch.FloatTensor of size 100]), 'delta': Normal([torch.FloatTensor of size 100]), 'Z_0': Normal([torch.FloatTensor of size 100]), 'Z_1': Normal([torch.FloatTensor of size 100]), 'X_1': Normal([torch.FloatTensor of size 100]), 'Z_2': Normal([torch.FloatTensor of size 100]), 'X_2': Normal([torch.FloatTensor of size 100]), 'Z_3': Normal([torch.FloatTensor of size 100]), 'X_3': Normal([torch.FloatTensor of size 100]), 'Z_4': Normal([torch.FloatTensor of size 100]), 'X_4': Normal([torch.FloatTensor of size 100]), 'Z_5': Normal([torch.FloatTensor of size 100]), 'X_5': Normal([torch.FloatTensor of size 100]), 'Z_6': Normal([torch.FloatTensor of size 100]), 'X_6': Normal([torch.FloatTensor of size 100]), 'Z_7': Normal([torch.FloatTensor of size 100]), 'X_7': Normal([torch.FloatTensor of size 100]), 'Z_8': Normal([torch.FloatTensor of size 100]), 'X_8': Normal([torch.FloatTensor of size 100]), 'Z_9': Normal([torch.FloatTenso

In [18]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1]), 'X_11': Normal([torch.Flo

In [19]:
for t in range(1, zs.shape[0]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.286538
SMC MSE at time 2: 0.709541
SMC MSE at time 3: 1.400701
SMC MSE at time 4: 1.791419
SMC MSE at time 5: 0.001446
SMC MSE at time 6: 1.247251
SMC MSE at time 7: 0.254145
SMC MSE at time 8: 0.125905
SMC MSE at time 9: 1.110598
SMC MSE at time 10: 5.331236
SMC MSE at time 11: 8.194860
SMC MSE at time 12: 1.342067
SMC MSE at time 13: 0.856928
SMC MSE at time 14: 0.840418
SMC MSE at time 15: 0.172030
SMC MSE at time 16: 0.220600
SMC MSE at time 17: 0.025455
SMC MSE at time 18: 2.123471
SMC MSE at time 19: 0.003576
SMC MSE at time 20: 0.915548
SMC MSE at time 21: 0.050696
SMC MSE at time 22: 1.203580
SMC MSE at time 23: 0.060985
SMC MSE at time 24: 0.159433
SMC MSE at time 25: 1.702383
SMC MSE at time 26: 0.130577
SMC MSE at time 27: 0.979133
SMC MSE at time 28: 4.800941
SMC MSE at time 29: 2.006094
SMC MSE at time 30: 2.307485
SMC MSE at time 31: 1.220110
SMC MSE at time 32: 0.755011
SMC MSE at time 33: 0.039870
SMC MSE at time 34: 0.351824
SMC MSE at time 35: 1.9

In [20]:
elbos

tensor([-147.9525, -135.5695, -135.5695, -135.5695, -135.5695, -135.5695,
        -135.5695, -135.5695, -135.5695, -135.5695, -135.5695, -135.5695,
        -135.5695, -135.5695, -135.5695, -135.5695, -135.5695, -135.5695,
        -135.5695, -135.5695, -135.5695, -135.5695, -135.5695, -135.5695,
        -135.5695, -135.5695, -135.5695, -135.5695, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.5069, -130.5069, -130.5069, -130.5069,
        -130.5069, -130.5069, -130.506